In [ ]:
import os
import cv2
import numpy as np
import statistics as st
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense , Conv2D, Flatten, MaxPooling2D
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,jaccard_score

**As the 10m resolution training data is unavailable, so we have to mannually generated it. For this we are using the True color sentinel-2A image(tcl_image.tif) of the region given in the input dataset of the PS-20. We semi-automatically labelled the data into 6 classes represented by 6 different color code that we know using the Google Earth Engine and named the image as 'labelled_image'.
The approach is straightforward, we will divide both the images simultaneously into 32x32 sub-images such that, the class assigned to 32x32 sub-image of the True color image, corresponds to the class of respective 32x32 sub-image of the labelled_image. As we know that each pixel of 32x32 sub-image of the labelled_image belong to a known set of color code, so we will take mode of the color codes and assign class corresponding to that color .
 We are saving the sub-images of the True color images in their respective folders.This methodology can be ectended to 18 classes. Different locations and seasons data need to be processed similarly for accumulate a better trainig dataset.**

In [ ]:
#Load your TCl_sentinel image and labeled image here
tcl_image = cv2.imread('/content/drive/MyDrive/Aiwizards/sentinel_image_rgb2.tif')
labeled_image = cv2.imread('/content/drive/MyDrive/Aiwizards/classified_image_rgb02.tif', cv2.IMREAD_UNCHANGED)  # Replace with your labeled image

# Define class names
class_names = ['Class0_BlackWaste', 'Class1_Water', 'Class2_Buildup', 'Class3_Barrenland', 'Class4_Cropland', 'Class5_Vegetation']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Function to extract subimages and save them to corresponding folders
def extract_and_save_subimages(tcl_image, labeled_image,output_dir,output_dir_outlier, subimage_size=(32, 32, 3)):
    # Create output directories if they don't exist
    for class_name in class_names:
        class_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
    for class_name in class_names:
        class_dir = os.path.join(output_dir_outlier, class_name)
        os.makedirs(class_dir, exist_ok=True)
    # Get the height and width of the images
    height, width, _ = tcl_image.shape

    # Iterate through the images and extract subimages
    for y in range(0, height - subimage_size[0] + 1, subimage_size[0]):
        for x in range(0, width - subimage_size[1] + 1, subimage_size[1]):
            subimage_tcl = tcl_image[y:y + subimage_size[0], x:x + subimage_size[1]]
            subimage_label = labeled_image[y:y + subimage_size[0], x:x + subimage_size[1]]

            # Determine the class based on the mode of pixel values in the labeled subimage
            pixel_color = []
            for i in range(subimage_size[0]):
                for j in range(subimage_size[1]):
                    if np.array_equal(subimage_label[i][j], [0, 0, 0]):
                        pixel_color.append(0)
                    elif np.array_equal(subimage_label[i][j], [255, 0, 0]):
                        pixel_color.append(1)
                    elif np.array_equal(subimage_label[i][j], [255, 255, 255]):
                        pixel_color.append(2)
                    elif np.array_equal(subimage_label[i][j], [0, 0, 255]):
                        pixel_color.append(3)
                    elif np.array_equal(subimage_label[i][j], [85, 255, 85]):
                        pixel_color.append(4)
                    elif np.array_equal(subimage_label[i][j], [0, 255, 0]):
                        pixel_color.append(5)
                    else:
                        print(subimage_label[i][j])

            if(st.mode(pixel_color)==0 and ((pixel_color.count(st.mode(pixel_color))*100)/1024)>30):
                class_index = st.mode(pixel_color)
                class_name = class_names[class_index]
                output_path = os.path.join(output_dir, class_name, f"subimage_{y}_{x}.tif")
                cv2.imwrite(output_path, subimage_tcl)
            if(st.mode(pixel_color)!=0 and ((pixel_color.count(st.mode(pixel_color))*100)/1024)>50):
                class_index = st.mode(pixel_color)
                class_name = class_names[class_index]
                output_path = os.path.join(output_dir, class_name, f"subimage_{y}_{x}.tif")
                cv2.imwrite(output_path, subimage_tcl)
            if (st.mode(pixel_color) == 0 and (((pixel_color.count(st.mode(pixel_color)) * 100) / 1024) <= 30 or ((pixel_color.count(st.mode(pixel_color)) * 100) / 1024) == 100)):
                class_index = st.mode(pixel_color)
                class_name = class_names[class_index]
                output_path = os.path.join(output_dir_outlier, class_name, f"subimage_{y}_{x}.tif")
                cv2.imwrite(output_path, subimage_tcl)
            if (st.mode(pixel_color) != 0 and ((pixel_color.count(st.mode(pixel_color)) * 100) / 1024) <= 50):
                class_index = st.mode(pixel_color)
                class_name = class_names[class_index]
                output_path = os.path.join(output_dir_outlier, class_name, f"subimage_{y}_{x}.tif")
                cv2.imwrite(output_path, subimage_tcl)


# Call the function to extract and save subimages
output_dir="train_subimages32" #path where the sub-images divided be stored
output_dir_outlier="tain_subimage32_outlier"
extract_and_save_subimages(tcl_image, labeled_image,output_dir,output_dir_outlier)



In [ ]:
# Function to create and train the CNN model
import matplotlib.pyplot as plt

# Function to create and train the CNN model
def create_and_train_model(input_shape, num_classes, class_weights, train_dir, test_dir, epochs=65):
    # Create the CNN model
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', input_shape=input_shape, activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(Flatten())
    model.add(Dense(768, activation='relu'))
    model.add(Dense(384, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


    # Use the ImageDataGenerator for data augmentation and preprocessing
    datagen =  tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255, validation_split=0.15)

    # Split the data into training, validation, and testing sets
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=input_shape[:2],
        batch_size=1000,
        class_mode='categorical',
        shuffle=True,
        subset='training',
        seed=40
    )
    validation_generator = datagen.flow_from_directory(
        train_dir,
        target_size=input_shape[:2],
        batch_size=1000,
        class_mode='categorical',
        shuffle=False,
        subset='validation',
        seed=40
    )

    # Train the model with validation data
    history = model.fit(train_generator, epochs=epochs, class_weight=class_weights, validation_data=validation_generator)

    # Save the model weights to a file
    model.save_weights('/content/drive/MyDrive/Aiwizards/Model_weightsANDtaining_plots/model_weights04_final.h5')

    # Alternatively, you can have a separate directory for testing
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape[:2],
        batch_size=200,
        class_mode='categorical',
        shuffle=False,
        seed=40
    )


    test_results = model.evaluate(test_generator)
    print("Test Loss:", test_results[0])
    print("Test Accuracy:", test_results[1])

    # Generate predictions for the testing set
    y_true = test_generator.classes
    y_pred_prob = model.predict(test_generator)
    y_pred = tf.argmax(y_pred_prob, axis=1).numpy()


    # Calculate and print additional metrics
    accuracy = accuracy_score(y_true, y_pred)
    confusion_mat = confusion_matrix(y_true, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print("Confusion Matrix:\n", confusion_mat)

    # Calculate and print F1 Score, Precision, Recall, and IoU
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    iou = jaccard_score(y_true, y_pred, average='weighted')

    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"IoU: {iou:.4f}")

    # Plot training and validation loss
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Save the plots to a file
    plt.savefig('/content/drive/MyDrive/Aiwizards/Model_weightsANDtaining_plots/training_plots04_final.png')
    plt.show()


    return model

# Path to the directory containing the training images
train_dir = "/content/drive/MyDrive/Aiwizards/train_subimages32.zip"
test_dir="/content/drive/MyDrive/Aiwizards/test_subimages32.zip"
# Number of classes
num_classes = len(class_names)

# Calculate class weights for imbalanced data
class_counts = [len(os.listdir(os.path.join(train_dir, class_name))) for class_name in class_names]
total_samples = sum(class_counts)
class_weights = {i: total_samples / (num_classes * class_counts[i]) for i in range(num_classes)}
print(class_weights)
# Model input shape and subimage size
input_shape = (32, 32, 3)
subimage_size = (32, 32, 3)

# Create and train the model
model = create_and_train_model(input_shape, num_classes, class_weights, train_dir,test_dir)


The testing data has 6 folders representing each class testing data. The testing data have been generated in the same way as the training data.

In [ ]:

###USER END TESTING
# Define class names
class_names = ['Class0_BlackWaste', 'Class1_Water', 'Class2_Buildup', 'Class3_Barrenland', 'Class4_Cropland', 'Class5_Vegetation']
num_classes = len(class_names)
input_shape = (32, 32, 3)
subimage_size = (32, 32, 3)
model = Sequential()
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', input_shape=input_shape, activation='relu'))
model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
model.add(Flatten())
model.add(Dense(768, activation='relu'))
model.add(Dense(384, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights('model_weights04_final.h5')
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Function to create an image with labeled subimages and evaluate on test data
def create_labeled_image_and_evaluate(model, subimage_size, testing_map, test_dir):
    ##When a large image of dimention like 1920X10280 given-

    # Load the large image
    large_image_path =testing_map  # Replace with the path to your large image
    large_image = cv2.imread(large_image_path)
    print(type(large_image))
    labeled_image = np.zeros((large_image.shape[0], large_image.shape[1], 3), dtype=np.uint8)
    large_image = cv2.resize(large_image, (large_image.shape[1] // subimage_size[1] * subimage_size[1],
                                           large_image.shape[0] // subimage_size[0] * subimage_size[0]))

    # Initialize the labeled image
    labeled_image = np.zeros_like(large_image)

    # Divide the large image into sub-images and make predictions
    for y in range(0, large_image.shape[0], subimage_size[0]):
        for x in range(0, large_image.shape[1], subimage_size[1]):
            subimg = large_image[y:y + subimage_size[0], x:x + subimage_size[1]]
            subimg = cv2.resize(subimg, (subimage_size[1], subimage_size[0]))

            # Predict the class of the subimage
            pred_class = model.predict(np.expand_dims(subimg, axis=0))
            pred_class = np.argmax(pred_class)

            # Assign RGB values based on the predicted class
            rgb_value = class_rgb_values[pred_class]

            # Fill the corresponding region in the labeled image
            labeled_image[y:y + subimage_size[0], x:x + subimage_size[1]] = rgb_value

    # Save the labeled image
    cv2.imwrite('labelled.png', labeled_image)
    '''

    # Generate the Classified Map, if the input is in same fiormat like we save the sub-images while generating the trainig data
    for class_name in class_names:
        class_dir = os.path.join(test_dir, class_name)
        for filename in os.listdir(class_dir):
            img = cv2.imread(os.path.join(class_dir, filename))
            img = cv2.resize(img, (subimage_size[1], subimage_size[0]))

            # Extract coordinates from the filename
            y, x = map(int, filename.split('_')[1:3])

            # Predict the class of the subimage
            pred_class = model.predict(np.expand_dims(img, axis=0))
            pred_class = np.argmax(pred_class)

            # Assign RGB values based on the predicted class
            rgb_value = class_rgb_values[pred_class]

            # Fill the corresponding region in the labeled image
            labeled_image[y:y + subimage_size[0], x:x + subimage_size[1]] = rgb_value

    # Save the labeled image to Google Drive
    cv2.imwrite(testing_map, labeled_image)
    '''

    #Evaluation

    # Use the trained model for predictions on the test dataset
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=subimage_size[:2],
        batch_size=1000,
        class_mode='categorical',
        shuffle=False
    )

    # Predict the classes
    predictions = model.predict(test_generator)

    # Convert predictions to class labels
    predicted_labels = np.argmax(predictions, axis=1)

    # Get the true labels from the test dataset
    true_labels = test_generator.classes

    # Calculate evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')

    # Confusion Matrix for IoU calculation
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    intersection = np.diag(conf_matrix)
    union = np.sum(conf_matrix, axis=0) + np.sum(conf_matrix, axis=1) - np.diag(conf_matrix)
    iou = np.mean(intersection / union)

    # Print the metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"IoU: {iou:.4f}")


# RGB values for each class
class_rgb_values = {
    0: [0, 0, 0],
    1: [85, 255, 255],
    2: [255, 255, 255],
    3: [255, 0, 0],
    4: [85, 255, 85],
    5: [0, 255, 0]
}

# Create the labeled image using the trained model and evaluate on test data
testing_map ='/content/drive/MyDrive/Aiwizards/sentinel_image_rgb2.tif'
test_dir = "/content/drive/MyDrive/Aiwizards/train_subimages32.zip"
create_labeled_image_and_evaluate(model, subimage_size, testing_map, test_dir)